# DUC Datathon: EDA on DUCs & Classification / Regression Models for DUC prediction.

## WorkFlow
    - Load Well data with time series DUC information
    - EDA Histograms, Box Plots etc.
    - Parse data into Train / Test set and a unseen validation set
    - Build a model to investigate the applicabiity to predict the duration a well will be a DUC.  Predict on a subset of    the entire time series. 
        - Run variations with & without TVD.
     - If successful, train a model on the first 60% of the time series and predict the remaining 40% of the time series to see if DUC inventory can be predicted with future area drilling plans.  
        - Train and forecast on projected depth, then compare with Train on actual Total Depth, and predict on Projected    Depth to mimic information available on planned, but undrilled wells.

In [ ]:
# Import libraries
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
from datetime import timedelta
import matplotlib.ticker as ticker
%matplotlib inline

In [ ]:
Wells = pd.read_csv('Wells_with_DUC_Time_Series.csv')

In [ ]:
Wells.head()

In [ ]:
Wells.drop(['Unnamed: 0','Unnamed: 0.1'], axis =1, inplace = True)

In [ ]:
# Convert Date Features in Wells to datetime
date_cols = ['SpudDate', 'FinalDrillDate','StatusDate', 'Early_Comp', 'Late_Comp', 'First_Prod', 'Last_Prod', 'Inferred_Comp_Date']
for col in date_cols:
    Wells[col] = pd.to_datetime(Wells[col], infer_datetime_format=True)

## Load a copy of Wells to use as a placeholder for model prediction and comparison on entire Cumulative DUCs

In [ ]:
df_R = pd.read_csv('Wells_with_DUC_Time_Series.csv')

In [ ]:
df_R.drop(['Unnamed: 0','Unnamed: 0.1'], axis =1, inplace = True)

In [ ]:
# Convert Date Features in Wells to datetime
date_cols = ['SpudDate', 'FinalDrillDate','StatusDate', 'Early_Comp', 'Late_Comp', 'First_Prod', 'Last_Prod', 'Inferred_Comp_Date']
for col in date_cols:
    df_R[col] = pd.to_datetime(df_R[col], infer_datetime_format=True)

In [ ]:
Wells.columns[0:40]

In [ ]:
Wells[['WellType', 'WellTypeStandardised',
       'KBElevation', 'TotalDepth', 'SpudDate', 'FinalDrillDate', 'RigReleaseDate',
       'DaysDrilling', 'TVD', 'WellProfile', 'Formation_C', 'Comp_Top', 'Early_Comp',
       'Comp_Base', 'TotalProd(BOE)','MonthsDUC_Status']].describe()

In [ ]:
sum(Wells['MonthsDUC_Status']==0), sum(Wells['MonthsDUC_Status']==1), sum(Wells['MonthsDUC_Status']==2), sum(Wells['MonthsDUC_Status']>2)

In [ ]:
plt.figure(figsize=(12,5)) 
sns.distplot(Wells['MonthsDUC_Status'], bins = 31, color = 'tab:blue', label = 'DUC Months - All Wells', kde = False)
sns.distplot(Wells['MonthsDUC_Status'][Wells['Type_C']==0], bins = 31, color = 'tab:red', label = 'DUC Months - GAS', kde = False)
sns.distplot(Wells['MonthsDUC_Status'][Wells['Type_C']==1], bins = 31, color = 'tab:green', label = 'DUC Months - OIL', kde = False)

plt.title('Histogram of Total Months a Well is a DUC')
plt.grid()
plt.legend()
plt.show()

In [ ]:
plt.figure(figsize=(12,5)) 
sns.distplot(Wells['MonthsDUC_Status'][Wells['Type_C']==0], bins = 31, color = 'tab:red', label = 'DUC Months - GAS', kde = False)
sns.distplot(Wells['MonthsDUC_Status'][Wells['Type_C']==1], bins = 31, color = 'tab:green', label = 'DUC Months - OIL', kde = False)

plt.title('Histogram of Total Months that a Well is a DUC, GAS & OIL')
plt.grid()
plt.legend()
plt.show()

In [ ]:
plt.figure(figsize=(12,5)) 
sns.distplot(Wells['MonthsDUC_Status'][Wells['Formation_C']==2], bins = 31, color = 'tab:green', label = 'DUC Months - Montney', kde = False)

plt.title('Histogram of Total Months a Well is a DUC')
plt.xlim(0,15)
plt.grid()
plt.legend()
plt.show()

In [ ]:
plt.figure(figsize=(12,5)) 
sns.distplot(Wells['MonthsDUC_Status'][Wells['Formation_C']==2][Wells['Type_C']==1], bins = 31, color = 'tab:green', label = 'DUC Months - Montney OIL', kde = False)
sns.distplot(Wells['MonthsDUC_Status'][Wells['Formation_C']==3][Wells['Type_C']==1], bins = 31, color = 'tab:red', label = 'DUC Months - Viking OIL', kde = False)
sns.distplot(Wells['MonthsDUC_Status'][Wells['Formation_C']==0][Wells['Type_C']==1], bins = 31, color = 'tab:blue', label = 'DUC Months - Cardium OIL', kde = False)
sns.distplot(Wells['MonthsDUC_Status'][Wells['Formation_C']==1][Wells['Type_C']==1], bins = 31, color = 'orange', label = 'DUC Months - Duvernay OIL', kde = False)

plt.title('Histogram of Total Months an OIL Well is a DUC')
plt.xlim(0,15)
plt.grid()
plt.legend()
plt.show()

### Get a stacked histogram binned by DUC-months with colors matching Tableau Presentation color scheme

In [ ]:
Cardium = []
Duvernay = []
Montney = []
Viking = []

Names = [Cardium, Duvernay, Montney, Viking]
for Form in range(4):
    for DucMonth in range(31):
        val = len(Wells['MonthsDUC_Status'][Wells['MonthsDUC_Status'] == DucMonth][Wells['Formation_C']==Form])
        Names[Form].append(val)


In [ ]:
plt.figure(figsize=(12,5))
plt.bar(range(31), Names[3], color = 'gold', label = "Viking")
plt.bar(range(31), Names[2], color = 'purple', label = 'Montney')
plt.bar(range(31), Names[0], color = 'b', label = 'Cardium')
plt.bar(range(31), Names[1], color = 'orange', label = 'Duvernay')
plt.title('Histogram of Total Months a Well is a DUC, by Formation', fontsize = 16, weight = 'bold')
plt.xlim(-1,25)
plt.legend()
plt.grid()
plt.show()

### Although the values are much lower for months 3 and above than for months 0 and 1, the number of wells are still significant and contribute to the ongoing inventory of DUCs.

In [ ]:
# Display the values for Montney for DUC-Months 0 to 31
Montney

In [ ]:
plt.figure(figsize=(12,5)) 
sns.distplot(Wells['MonthsDUC_Status'][Wells['Formation_C']==2][Wells['Type_C']==0], bins = 31, color = 'tab:green', label = 'DUC Months - Montney GAS', kde = False)
sns.distplot(Wells['MonthsDUC_Status'][Wells['Formation_C']==0][Wells['Type_C']==0], bins = 31, color = 'tab:blue', label = 'DUC Months - Cardium GAS', kde = False)
sns.distplot(Wells['MonthsDUC_Status'][Wells['Formation_C']==1][Wells['Type_C']==0], bins = 31, color = 'orange', label = 'DUC Months - Duvernay GAS', kde = False)
sns.distplot(Wells['MonthsDUC_Status'][Wells['Formation_C']==3][Wells['Type_C']==0], bins = 31, color = 'tab:red', label = 'DUC Months - Viking GAS', kde = False)

plt.title('Histogram of Total Months a GAS Well is a DUC')
plt.xlim(0,15)
plt.grid()
plt.legend()
plt.show()

In [ ]:
plt.figure(figsize=(12,5)) 
sns.distplot(Wells['MonthsDUC_Status'][Wells['Type_C']==1], bins = 31, color = 'tab:green', label = 'DUC Months', kde = False)
plt.title('Histogram of Total Months a OIL Well is a DUC')
plt.grid()
plt.show()

In [ ]:
colors_pallette =['blue', 'orange', 'purple', 'gold']

fig, (ax1, ax2) = plt.subplots(1, 2, figsize = (12, 5), sharey = False)
ax1=sns.boxplot(Wells['Formation_C'][Wells['Type_C']==0], Wells['MonthsDUC_Status'][Wells['Type_C']==0],
                 ax = ax1)
ax2=sns.boxplot(Wells['Formation_C'][Wells['Type_C']==1], Wells['MonthsDUC_Status'][Wells['Type_C']==1],
                 ax = ax2)

ax1.set_title('GAS Well Box Plot DUC Months by Formation')
ax2.set_title('OIL Well Box Plot DUC Months by Formation')
plt.show()

In [ ]:
#Plot the same info in a Boxen plot to compare the visualization. 

fig, (ax1, ax2) = plt.subplots(1, 2, figsize = (12, 5), sharey = False)
ax1=sns.boxenplot(Wells['Formation_C'][Wells['Type_C']==0], Wells['MonthsDUC_Status'][Wells['Type_C']==0], ax = ax1)
ax2=sns.boxenplot(Wells['Formation_C'][Wells['Type_C']==1], Wells['MonthsDUC_Status'][Wells['Type_C']==1], ax = ax2)

ax1.set_title('GAS Well Boxen Plot DUC Months by Formation')
ax2.set_title('OIL Well Boxen Plot DUC Months by Formation')

plt.show()

In [ ]:
#Plot the same info in a Violin plot to compare the visualization.

fig, (ax1, ax2) = plt.subplots(1, 2, figsize = (12, 5), sharey = False)
ax1=sns.violinplot(Wells['Formation_C'][Wells['Type_C']==0], Wells['MonthsDUC_Status'][Wells['Type_C']==0], ax = ax1)
ax2=sns.violinplot(Wells['Formation_C'][Wells['Type_C']==1], Wells['MonthsDUC_Status'][Wells['Type_C']==1], ax = ax2)

ax1.set_title('GAS Well Violin Plot DUC Months by Formation')
ax2.set_title('OIL Well Violin Plot DUC Months by Formation')

plt.show()

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize = (12, 5), sharey = False)
ax1=sns.boxplot(Wells['WellProfile'][Wells['Type_C']==0], Wells['MonthsDUC_Status'][Wells['Type_C']==0], ax = ax1)
ax2=sns.boxplot(Wells['WellProfile'][Wells['Type_C']==1], Wells['MonthsDUC_Status'][Wells['Type_C']==1], ax = ax2)

ax1.set_title('GAS Well Box Plot DUC Months by Profile')
ax2.set_title('OIL Well Box Plot DUC Months by Profile')

plt.show()

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize = (12, 5), sharey = False)
ax1=sns.boxenplot(Wells['WellProfile'][Wells['Type_C']==0], Wells['MonthsDUC_Status'][Wells['Type_C']==0], ax = ax1)
ax2=sns.boxenplot(Wells['WellProfile'][Wells['Type_C']==1], Wells['MonthsDUC_Status'][Wells['Type_C']==1], ax = ax2)

ax1.set_title('GAS Well Boxen Plot DUC Months by Profile')
ax2.set_title('OIL Well Boxen Plot DUC Months by Profile')

plt.show()

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize = (12, 5), sharey = False)
ax1=sns.violinplot(Wells['WellProfile'][Wells['Type_C']==0], Wells['MonthsDUC_Status'][Wells['Type_C']==0], ax = ax1)
ax2=sns.violinplot(Wells['WellProfile'][Wells['Type_C']==1], Wells['MonthsDUC_Status'][Wells['Type_C']==1], ax = ax2)

ax1.set_title('GAS Well Vioilin Plot DUC Months by Profile')
ax2.set_title('OIL Well Violin Plot DUC Months by Profile')

plt.show()

### Box Plots by Field for each Formation

In [ ]:
plt.figure(figsize=(15,5))
sns.boxplot(Wells['Field'][Wells['Type_C']<2][Wells['Formation_C']==0],
                Wells['MonthsDUC_Status'][Wells['Type_C']<2][Wells['Formation_C']==0])
plt.title('GAS & OIL Well Box Plot DUC Months for Cardium')
plt.xticks(rotation=45)
plt.show()

In [ ]:
#Boxen plot variation

plt.figure(figsize=(15,5))
sns.boxenplot(Wells['Field'][Wells['Type_C']<2][Wells['Formation_C']==0],
                Wells['MonthsDUC_Status'][Wells['Type_C']<2][Wells['Formation_C']==0])
plt.title('GAS & OIL Well Boxen Plot DUC Months for Cardium')
plt.xticks(rotation=45)
plt.show()

In [ ]:
plt.figure(figsize=(15,5))
sns.boxplot(Wells['Field'][Wells['Type_C']<2][Wells['Formation_C']==1],
                Wells['MonthsDUC_Status'][Wells['Type_C']<2][Wells['Formation_C']==1])
plt.title('GAS & OIL Well Box Plot DUC Months for Duvernay')
plt.xticks(rotation=45)
plt.show()

In [ ]:
plt.figure(figsize=(15,5))
sns.boxplot(Wells['Field'][Wells['Type_C']<2][Wells['Formation_C']==2],
                Wells['MonthsDUC_Status'][Wells['Type_C']<2][Wells['Formation_C']==2])
plt.title('GAS & OIL Well Box Plot DUC Months for Montney')
plt.xticks(rotation=45)
plt.show()

In [ ]:
plt.figure(figsize=(15,5))
sns.boxplot(Wells['Field'][Wells['Type_C']<2][Wells['Formation_C']==3],
                Wells['MonthsDUC_Status'][Wells['Type_C']<2][Wells['Formation_C']==3])
plt.title('GAS & OIL Well Box Plot DUC Months for Viking')
plt.xticks(rotation=45)
plt.show()

### Load Classification and Regression Models

### First runs will be use all data and then checked on how the cumulative DUCs curve changes

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, classification_report, fbeta_score, accuracy_score, roc_auc_score, roc_curve, auc
from sklearn.metrics import r2_score, mean_squared_error

# DUC Prediction

### The Cumulative DUC inventory in logically driven by the wells that are DUCs for a longer priod of time

### Every potentially productive well is a DUC, for a day, a week, a month or longer .  The DUC inventory won't be changed in the long term by the revolving inventory of short terms DUCs, some adding while others are subtracting.

### Decisions by the operator that any well will remain a DUC are determined by a number of outside factors such as commodity price, Operator financial status, land retention requirements and more.

### The accuracy of predicting the length of time a specific well will be a DUC may be suspect.  However, can we aggregate a well by well prediction and predict the total number of DUC-months on a group of wells or the number of DUC-months for a group of wells over a nominal number of 1 to 3 months.

## Build classification and regression models and substitute the x_test results into the dataset and plot the Cumulative DUC charts with & without the model

In [ ]:
# Get minimim Final Drill Date and find a starting point to change the drill date to number of days from a common date
# so we can use the FINALDRILLDATE in the model
min(Wells['FinalDrillDate'])

In [ ]:
# CONVERT FINALDRILLDATE TO DAYS FROM Jan 1 2014 FOR ML MODEL
Wells['EndDrillDate_InDays'] = (Wells['FinalDrillDate']-pd.to_datetime('2014-01-01')).apply(lambda x: float(x.days))


### Group the DUC Months into classes 0,1,2,3,4 months and 5 for >5 months

In [ ]:
Wells['DUC_CountGroup'] = 5 -  1*(Wells['MonthsDUC_Status']<5) - 1*(Wells[
    'MonthsDUC_Status']<4) - 1*(Wells['MonthsDUC_Status']<3) - 1 *(Wells['MonthsDUC_Status']<2) - 1 *(Wells['MonthsDUC_Status']<1)

In [ ]:
Wells['DUC_CountGroup'].value_counts()

In [ ]:
Wells['MonthsDUC_Status'].value_counts()

### Get Stats on the classification Groups

In [ ]:
# ALL WELLS
Wells['MonthsDUC_Status'][Wells['DUC_CountGroup']==5].describe()

In [ ]:
# GAS Wells
Wells['MonthsDUC_Status'][Wells['DUC_CountGroup']==5][Wells['Type_C']==0].describe()

In [ ]:
# OIL Wells
Wells['MonthsDUC_Status'][Wells['DUC_CountGroup']==5][Wells['Type_C']==1].describe()

In [ ]:
# MEAN for GAS Wells by Formation 0,1,2,3 (Cardium, Duvernay, Montney, Viking)
for n in range(4):
    print('Mean for GAS wells class 5 for Formation ',n,' is',round(Wells['MonthsDUC_Status'][Wells['DUC_CountGroup']==5][Wells['Type_C']==0][Wells['Formation_C']==n].describe()[1], 3))

In [ ]:
# MEAN for OIL Wells by Formation 0,1,2,3 (Cardium, Duvernay, Montney, Viking)
for n in range(4):
    print('Mean for OIL wells class 5 for Formation ',n,' is',round(Wells['MonthsDUC_Status'][Wells['DUC_CountGroup']==5][Wells['Type_C']==1][Wells['Formation_C']==n].describe()[1], 3))

###  Use 7.45 for models except if run by Formation

## Get X set for ALL Wells - do not split by well type or formation

In [ ]:
X = Wells[['KBElevation','TVD', 'TotalDepth','EndDrillDate_InDays', 
        'Profile_C', 'Formation_C', 'Field_C', 'Type_C','MonthsDUC_Status','DUC_CountGroup']][Wells['TVD']>0][Wells['Formation_C']>=0]
X.shape

## Classification

In [ ]:
y = X['MonthsDUC_Status']
yC = X['DUC_CountGroup']
X.drop(['MonthsDUC_Status','DUC_CountGroup'], axis =1, inplace = True)
y.shape, yC.shape

In [ ]:
X.head()

In [ ]:
X_train,X_test, y_train, y_test = train_test_split(X,yC, stratify = yC, test_size=0.25, random_state=0)

In [ ]:
# CHECK SPLITS BY FORMATION
sum(X_train['Formation_C']==2), sum(X_test['Formation_C']==2), sum(X_train['Formation_C']==3), sum(X_test['Formation_C']==3) 

In [ ]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
n, d, crit = 175, 12, 'gini'
RFC = RandomForestClassifier(n_estimators = n, max_depth = d, criterion = crit, random_state = 0)
RFC.fit(X_train, y_train)

In [ ]:
y_pred = RFC.predict(X_test)
print(classification_report(y_test, y_pred))

In [ ]:
print(confusion_matrix(y_test, y_pred))

## Note that the confusion matrix is showing the model is not acurately predicting well by well DUC months.  However, the overall aggregated trend of number of wells with low DUC months and number of wells with high DUC months is better.

## One case where a well by well error could appear that doesn't affect the overal outcome this could happen is with multiple wells on a pad.  If one or two wells have many DUC months, the difference between those wells is slight and the prediction might be accurate on a pad basis but inaccurate on an individual well basis. 

In [ ]:
y_test.value_counts()

In [ ]:
YP=pd.Series(y_pred)
YP.value_counts()

### Look at y_test and y_pred total DUC months and for DUC months > 2 per well 

In [ ]:
y_test_SumMonths = sum((y_test==1)) + 2*sum((y_test==2)) + 3*sum((y_test==3)) + 4*sum((y_test==4)) + 7.45*sum((y_test==5))
y_test_SumMonths

In [ ]:
y_pred_SumMonths = sum((y_pred==1)) + 2*sum((y_pred==2)) + 3*sum((y_pred==3)) + 4*sum((y_pred==4)) + 7.45*sum((y_pred==5))
y_pred_SumMonths

In [ ]:
y_test_SumMonths3Over = 3*sum((y_test==3)) + 4*sum((y_test==4)) + 7.45*sum((y_test==5)) 
y_test_SumMonths3Over

In [ ]:
y_pred_SumMonths3Over = 3*sum((y_pred==3)) + 4*sum((y_pred==4)) + 7.45*sum((y_pred==5))
y_pred_SumMonths3Over

### Model is under predicting DUC Months, Compare Distributions

In [ ]:
plt.figure(figsize=(12,5)) 
sns.distplot(y_test, bins = 11, color = 'tab:green', label = 'y_test', kde = False)
sns.distplot(YP, bins = 11, color = 'tab:blue', label = 'y-pred', kde = False)


plt.title('Histogram y-test & y_pred')
#plt.xlim(0,15)
#plt.yscale('log')
plt.grid()
plt.legend()
plt.show()

## Substitue the predicted months for X_test into the Wells copy(df_R) then calculate and plot cumulative DUCs by month.

In [ ]:
# CHANGE RELEVANT EARLY COMPLETION DATES
vals = [0,1,2,3,4,7.45]  #Class 5 has a mean of 7.45 months

for j in range(len(y_test)):
    ID = df_R.iloc[y_test[j:j+1].index[0]][0]
    
    # CALCULATE THE PREDICTED MONTHS FROM THE CLASSIFICATION 0 TO 5
    val = y_pred[j]
    pred = vals[val]

    df_R['Early_Comp'][df_R['EPAssetsId']==ID] = df_R['FinalDrillDate'][df_R['EPAssetsId']==ID] + timedelta(days = int(pred*30.45))

In [ ]:
Months = ['2015-01-31', '2015-02-28',
       '2015-03-31', '2015-04-30', '2015-05-31', '2015-06-30', '2015-07-31',
       '2015-08-31', '2015-09-30', '2015-10-31', '2015-11-30', '2015-12-31',
       '2016-01-31', '2016-02-29', '2016-03-31', '2016-04-30', '2016-05-31',
       '2016-06-30', '2016-07-31', '2016-08-31', '2016-09-30', '2016-10-31',
       '2016-11-30', '2016-12-31', '2017-01-31', '2017-02-28', '2017-03-31',
       '2017-04-30', '2017-05-31', '2017-06-30', '2017-07-31', '2017-08-31',
       '2017-09-30', '2017-10-31', '2017-11-30', '2017-12-31', '2018-01-31',
       '2018-02-28', '2018-03-31', '2018-04-30', '2018-05-31', '2018-06-30',
       '2018-07-31', '2018-08-31', '2018-09-30', '2018-10-31', '2018-11-30',
       '2018-12-31', '2019-01-31', '2019-02-28', '2019-03-31', '2019-04-30',
       '2019-05-31', '2019-06-30', '2019-07-31', '2019-08-31', '2019-09-30',
       '2019-10-31', '2019-11-30', '2019-12-31', '2020-01-31']

In [ ]:
for Month in Months:
    df_R[Month] = 1*( df_R['FinalDrillDate']<=pd.to_datetime(Month) ) - 1*( df_R['Early_Comp']<=pd.to_datetime(Month) )

In [ ]:
header =['Model'] + Months
DUCs_Compare = pd.DataFrame(columns = header)
DUCs_Compare['Model']=['BaseCase', 'Classifier 1']
DUCs_Compare.head()

In [ ]:
#Load Base Case DUCs into df

for j in range(len(Months)):
    DUCs_Compare.iloc[0][j+1] = (sum(Wells[Months[j]]))
DUCs_Compare.shape

In [ ]:
#Load Classification results into df
for j in range(len(Months)):
    DUCs_Compare.iloc[1][j+1] = (sum(df_R[Months[j]]))
DUCs_Compare.head()
 

In [ ]:
fig, ax1 = plt.subplots(1, 1, figsize=(15, 8)) 
ax1.plot(DUCs_Compare.iloc[0,1:], label = DUCs_Compare.iloc[0,0], c = 'm', lw = 3)
ax1.plot(DUCs_Compare.iloc[1,1:], label = DUCs_Compare.iloc[1,0])

ax1.set_title('Cumulative DUCs by Month: Comparison of Base Case and DUC Prediction Models', fontsize = 16, weight = 'bold')
ax1.xaxis.set_major_locator(ticker.MaxNLocator(11))
ax1.xaxis.set_minor_locator(ticker.AutoMinorLocator())
ax1.grid()
ax1.set_ylim(0, 450)
plt.legend()
plt.show()



In [ ]:
DUCs_Compare.head()

# Regression

In [ ]:
# X FILTERED FOR WELL WITHOUT TVD
X = Wells[['KBElevation','TVD', 'TotalDepth','EndDrillDate_InDays', 
        'Profile_C', 'Formation_C', 'Field_C', 'Type_C','MonthsDUC_Status','DUC_CountGroup']][Wells['TVD']>0][Wells['Formation_C']>=0]
X.shape

In [ ]:
y = X['MonthsDUC_Status']
yC = X['DUC_CountGroup']
X.drop(['MonthsDUC_Status','DUC_CountGroup'], axis =1, inplace = True)
y.shape, yC.shape

In [ ]:
X_train,X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=0)

In [ ]:
len(X_test)

In [ ]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
n, d, mf = 175, 26, 'log2'
RFR = RandomForestRegressor(n_estimators = n, max_depth = d, max_features = mf, random_state = 0)
RFR.fit(X_train, y_train)

In [ ]:
y_pred = RFR.predict(X_test)
RFR_R2 = r2_score(y_test, y_pred)
MSE = mean_squared_error(y_test, y_pred)
RMSE = np.sqrt(MSE)
print(n, " ests & " ,d, " max depth,", mf, "max features, R2 for RFR is: ", RFR_R2)
print(n, " ests & ", d, " max depth,", mf, "max features, RMSE RFR is: ", RMSE)

In [ ]:
plt.scatter(y_test, y_pred)
plt.show()

In [ ]:
y_test_SumMonths = sum(y_test)
y_test_SumMonths

In [ ]:
y_pred_SumMonths = sum(y_pred)
y_pred_SumMonths

In [ ]:
YP = pd.Series(y_pred)

In [ ]:
y_test.describe()

In [ ]:
YP.describe()

In [ ]:
# COMPARE TOTAL MONTHS AND MONTHS FOR WELLS OVER 2 MONTHS
sum(y_test), sum(y_pred),  sum(y_test[y_test>2]), sum(YP[YP>2]) 

## Aggregated totals are similar, despite the low R2 score

### Look at counts over 2 months and the mean of actuals and results over 2 months and the product

In [ ]:
TS, TM, PS, PM  = sum(y_test>2), np.mean(y_test[y_test>2]),sum(YP>2),  np.mean(YP[YP>2])
TS, round(TM,2), (TS*TM), PS, round(PM,3), round(PS*PM,1)

## Model is under predicting, but much closer than the classification, Look at the Distrubutions

In [ ]:
plt.figure(figsize=(12,5)) 
sns.distplot(y_test, bins = 15, color = 'tab:green', label = 'y_test', kde = False)
sns.distplot(YP, bins = 15, color = 'tab:blue', label = 'y-pred', kde = False)


plt.title('Regression Model 1 Histogram y-test & y_pred')
#plt.xlim(0,15)
#plt.yscale('log')
plt.grid()
plt.legend()
plt.show()

# Now substitue the prediction into Wells by changing the Early Completion Date and then recalculate DUC status by month.  First Reset Early Comp Date

In [ ]:
df_R['Early_Comp'] = Wells['Early_Comp']

In [ ]:
# CHANGE RELEVANT EARLY COMPLETION DATES - REGRESSION RESULTS ARE DIRECT MONTHS

for j in range(len(y_test)):
    ID = df_R.iloc[y_test[j:j+1].index[0]][0]
    
    # Y_PRED VALUES ARE FORECASTED AS MONTHS NOT CLASSES, SO NO CONVERSION REQUIRED
    pred = y_pred[j]
    #pred = vals[val]

    df_R['Early_Comp'][df_R['EPAssetsId']==ID] = df_R['FinalDrillDate'][df_R['EPAssetsId']==ID] + timedelta(days = int(pred*30.45))

In [ ]:
for Month in Months:
    df_R[Month] = 1*( df_R['FinalDrillDate']<=pd.to_datetime(Month) ) - 1*( df_R['Early_Comp']<=pd.to_datetime(Month) )

In [ ]:
DUCs_by_Month_with_y_pred = ['Regression 1']
for Month in Months:
    DUCs_by_Month_with_y_pred.append(sum(df_R[Month]))

In [ ]:
DUCs_Compare.loc[2] = DUCs_by_Month_with_y_pred
DUCs_Compare.head()

In [ ]:
fig, ax1 = plt.subplots(1, 1, figsize=(15, 8)) 
ax1.plot(DUCs_Compare.iloc[0,1:], label = DUCs_Compare.iloc[0,0], c = 'm', lw = 3)
ax1.plot(DUCs_Compare.iloc[1,1:], label = DUCs_Compare.iloc[1,0])
ax1.plot(DUCs_Compare.iloc[2,1:], label = DUCs_Compare.iloc[2,0])

ax1.set_title('Cumulative DUCs by Month: Comparison of Base Case and DUC Prediction Models', fontsize = 16, weight = 'bold')
ax1.xaxis.set_major_locator(ticker.MaxNLocator(11))
ax1.xaxis.set_minor_locator(ticker.AutoMinorLocator())
ax1.set_ylim(0,450)
ax1.grid()
plt.legend()
plt.show()


In [ ]:
# COMPARE DELTA TO BASE CASE AND CALCULTAE PERCENT ERROR FOR REGRESSION
Err = 0
month = '2015-01-31'
for Month in Months:
    delta_err = abs((DUCs_Compare[Month][0] - DUCs_Compare[Month][2]) / DUCs_Compare[Month][0])
    if delta_err > Err:
        Err = delta_err
        month = Month
'Regression', round(Err, 3), Month

In [ ]:
# COMPARE DELTA TO BASE CASE AND CALCULTAE PERCENT ERROR FOR CLASSIFICATION
Err = 0
month = '2015-01-31'
for Month in Months:
    delta_err = abs( 1 - DUCs_Compare[Month][1]  / DUCs_Compare[Month][0] )
    if delta_err > Err:
        Err = delta_err
        month = Month
'Classification', round(Err, 3), Month

# The x-test size is about 1/6th of the total number of wells.  The max error is 11.8% in Jan 2020, 74 DUCs vs 87 actual for Regression and 12.9% in Jan 2020, 80 DUCs vs 87 for Classification

In [ ]:
DUCs_Compare.head()

In [ ]:
sum(DUCs_Compare.iloc[0,1:]),  sum(DUCs_Compare.iloc[1,1:]), sum(DUCs_Compare.iloc[2,1:])

## Retrain the models without TVD and Compare, then train the models on the first three years of data and predict the remaining 2 years.  Need to drop TVD to be able to predict on all wells going forward.

In [ ]:
X = Wells[['KBElevation','TotalDepth','EndDrillDate_InDays', 
        'Profile_C', 'Formation_C', 'Field_C', 'Type_C','MonthsDUC_Status','DUC_CountGroup']][Wells['Formation_C']>=0]
X.shape

## Classification

In [ ]:
y = X['MonthsDUC_Status']
yC = X['DUC_CountGroup']
X.drop(['MonthsDUC_Status','DUC_CountGroup'], axis =1, inplace = True)
y.shape, yC.shape

In [ ]:
X.head()

In [ ]:
X_train,X_test, y_train, y_test = train_test_split(X,yC, stratify = yC, test_size=0.25, random_state=0)

In [ ]:
# CHECK SPLITS BY FORMATION
sum(X_train['Formation_C']==2), sum(X_test['Formation_C']==2), sum(X_train['Formation_C']==3), sum(X_test['Formation_C']==3) 

In [ ]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
n, d, crit = 175, 12, 'gini'
RFC = RandomForestClassifier(n_estimators = n, max_depth = d, criterion = crit, random_state = 0)
RFC.fit(X_train, y_train)

In [ ]:
y_pred = RFC.predict(X_test)
print(classification_report(y_test, y_pred))

In [ ]:
print(confusion_matrix(y_test, y_pred))

## Substitue the predicted months for X_test into the Wells copy(df_R) then calculate and plot cumulative DUCs by month.  Rest Early Comp Dates

In [ ]:
df_R['Early_Comp'] = Wells['Early_Comp']

In [ ]:
# CHANGE RELEVANT EARLY COMPLETION DATES
vals = [0,1,2,3,4,7.45]  #Class 5 has a mean of 7.45 months

for j in range(len(y_test)):
    ID = df_R.iloc[y_test[j:j+1].index[0]][0]
    
    # CALCULATE THE PREDICTED MONTHS FROM THE CLASSIFICATION 0 TO 5
    val = y_pred[j]
    pred = vals[val]

    df_R['Early_Comp'][df_R['EPAssetsId']==ID] = df_R['FinalDrillDate'][df_R['EPAssetsId']==ID] + timedelta(days = int(pred*30.45))

In [ ]:
for Month in Months:
    df_R[Month] = 1*( df_R['FinalDrillDate']<=pd.to_datetime(Month) ) - 1*( df_R['Early_Comp']<=pd.to_datetime(Month) )

In [ ]:
DUCs_by_Month_with_y_pred = ['Classification w/o TVD']
for Month in Months:
    DUCs_by_Month_with_y_pred.append(sum(df_R[Month]))

In [ ]:
DUCs_Compare.loc[3] = DUCs_by_Month_with_y_pred
DUCs_Compare.head()

In [ ]:
fig, ax1 = plt.subplots(1, 1, figsize=(15, 8)) 
ax1.plot(DUCs_Compare.iloc[0,1:], label = DUCs_Compare.iloc[0,0], c = 'm', lw = 3)
ax1.plot(DUCs_Compare.iloc[1,1:], label = DUCs_Compare.iloc[1,0])
ax1.plot(DUCs_Compare.iloc[2,1:], label = DUCs_Compare.iloc[2,0])
ax1.plot(DUCs_Compare.iloc[3,1:], label = DUCs_Compare.iloc[3,0])

ax1.set_title('Cumulative DUCs by Month: Comparison of Base Case and DUC Prediction Models', fontsize = 16, weight = 'bold')
ax1.xaxis.set_major_locator(ticker.MaxNLocator(11))
ax1.xaxis.set_minor_locator(ticker.AutoMinorLocator())
ax1.grid()
ax1.set_ylim(0, 450)
plt.legend()
plt.show()



In [ ]:
# X FILTERED FOR WELL WITHOUT TVD
X = Wells[['KBElevation','TotalDepth','EndDrillDate_InDays', 
        'Profile_C', 'Formation_C', 'Field_C', 'Type_C','MonthsDUC_Status','DUC_CountGroup']][Wells['Formation_C']>=0]
X.shape

In [ ]:
y = X['MonthsDUC_Status']
yC = X['DUC_CountGroup']
X.drop(['MonthsDUC_Status','DUC_CountGroup'], axis =1, inplace = True)
y.shape, yC.shape

In [ ]:
X_train,X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=0)

In [ ]:
len(X_test)

In [ ]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
n, d, mf = 175, 26, 'log2'
RFR = RandomForestRegressor(n_estimators = n, max_depth = d, max_features = mf, random_state = 0)
RFR.fit(X_train, y_train)

In [ ]:
y_pred = RFR.predict(X_test)
RFR_R2 = r2_score(y_test, y_pred)
MSE = mean_squared_error(y_test, y_pred)
RMSE = np.sqrt(MSE)
print(n, " ests & " ,d, " max depth,", mf, "max features, R2 for RFR is: ", RFR_R2)
print(n, " ests & ", d, " max depth,", mf, "max features, RMSE RFR is: ", RMSE)

In [ ]:
plt.scatter(y_test, y_pred)
plt.show()

In [ ]:
plt.figure(figsize=(12,5)) 
sns.distplot(y_test, bins = 15, color = 'tab:green', label = 'y_test', kde = False)
sns.distplot(y_pred, bins = 15, color = 'tab:blue', label = 'y-pred', kde = False)


plt.title('Regression Model 2 w/o TVD Histogram y-test & y_pred')
#plt.xlim(0,15)
#plt.yscale('log')
plt.grid()
plt.legend()
plt.show()

# Now substitue the prediction into Wells by changing the Early Completion Date and then recalculate DUC status by month.  Reset Early Comp Date.

In [ ]:
df_R['Early_Comp'] = Wells['Early_Comp']

In [ ]:
# CHANGE RELEVANT EARLY COMPLETION DATES - REGRESSION RESULTS ARE DIRECT MONTHS

for j in range(len(y_test)):
    ID = df_R.iloc[y_test[j:j+1].index[0]][0]
    
    # Y_PRED VALUES ARE FORECASTED AS MONTHS NOT CLASSES, SO NO CONVERSION REQUIRED
    pred = y_pred[j]
    #pred = vals[val]

    df_R['Early_Comp'][df_R['EPAssetsId']==ID] = df_R['FinalDrillDate'][df_R['EPAssetsId']==ID] + timedelta(days = int(pred*30.45))

In [ ]:
for Month in Months:
    df_R[Month] = 1*( df_R['FinalDrillDate']<=pd.to_datetime(Month) ) - 1*( df_R['Early_Comp']<=pd.to_datetime(Month) )

In [ ]:
DUCs_by_Month_with_y_pred = ['Regression 2 w/o TVD']
for Month in Months:
    DUCs_by_Month_with_y_pred.append(sum(df_R[Month]))

In [ ]:
DUCs_Compare.loc[4] = DUCs_by_Month_with_y_pred
DUCs_Compare.head()

In [ ]:
fig, ax1 = plt.subplots(1, 1, figsize=(15, 8)) 
ax1.plot(DUCs_Compare.iloc[0,1:], label = DUCs_Compare.iloc[0,0], c = 'm', lw = 3)
ax1.plot(DUCs_Compare.iloc[1,1:], label = DUCs_Compare.iloc[1,0])
ax1.plot(DUCs_Compare.iloc[2,1:], label = DUCs_Compare.iloc[2,0])
ax1.plot(DUCs_Compare.iloc[3,1:], label = DUCs_Compare.iloc[3,0])
ax1.plot(DUCs_Compare.iloc[4,1:], label = DUCs_Compare.iloc[4,0])

ax1.set_title('Cumulative DUCs by Month: Comparison of Base Case and DUC Prediction Models', fontsize = 16, weight = 'bold')
ax1.xaxis.set_major_locator(ticker.MaxNLocator(11))
ax1.xaxis.set_minor_locator(ticker.AutoMinorLocator())
ax1.set_ylim(0,450)
ax1.grid()
plt.legend()
plt.show()


In [ ]:
# COMPARE DELTA TO BASE CASE AND CALCULTAE PERCENT ERROR FOR REGRESSION - 2 w/o TVD
Err = 0
month = '2015-01-31'
for Month in Months:
    delta_err = abs((DUCs_Compare.loc[0][Month] - DUCs_Compare.loc[4][Month]) / DUCs_Compare.loc[0][Month])
    if delta_err > Err:
        Err = delta_err
        month = Month
'Regression', round(Err, 3), Month

In [ ]:
# COMPARE DELTA TO BASE CASE AND CALCULTAE PERCENT ERROR FOR CLASSIFICATION - 2 w/o TVD
Err = 0
month = '2015-01-31'
for Month in Months:
    delta_err = abs((DUCs_Compare.loc[0][Month] - DUCs_Compare.loc[3][Month]) / DUCs_Compare.loc[0][Month])
    if delta_err > Err:
        Err = delta_err
        month = Month
'Classification', round(Err, 3), Month

## These last 2 models are predicted on an x_test size of ~ 2600 wells (25%).  The maximum error at the low loint is equal or greater than this number, but the error is smaller at the peaks.

## These models have not been optimized.

## The Classification model w/o TVD has a lower maximum error than the regression model.  Use this when re-training up to the end of 2017.  This model performs well at the peaks.

In [ ]:
DUCs_Compare.head()

### Look at cumulative sum of DUC-Months for the models vs the actual - Base Case

In [ ]:
sum(DUCs_Compare.iloc[0,1:]),  sum(DUCs_Compare.iloc[1,1:]), sum(DUCs_Compare.iloc[2,1:]), sum(DUCs_Compare.iloc[3,1:]), sum(DUCs_Compare.iloc[4,1:] )

## Now investigate the performamce of training the models on the first three years of data and predict the remaining 2 years.  If successful, then a model can be used to predict furture DUC inventories with information on formation or area industry drilling plans.

In [ ]:
(pd.to_datetime('2017-12-31') - pd.to_datetime('2014-01-01')).days

In [ ]:
X = Wells[['KBElevation','TotalDepth','EndDrillDate_InDays', 
        'Profile_C', 'Formation_C', 'Field_C', 'Type_C','MonthsDUC_Status',
           'DUC_CountGroup']][Wells['EndDrillDate_InDays'] <= 1460]
X.shape

## Classification

In [ ]:
y = X['MonthsDUC_Status']
yC = X['DUC_CountGroup']
X.drop(['MonthsDUC_Status','DUC_CountGroup'], axis =1, inplace = True)
y.shape, yC.shape

In [ ]:
X.head()

In [ ]:
X_train,X_test, y_train, y_test = train_test_split(X,yC, stratify = yC, test_size=0.10, random_state=0)

In [ ]:
# CHECK SPLITS BY FORMATION
sum(X_train['Formation_C']==2), sum(X_test['Formation_C']==2), sum(X_train['Formation_C']==3), sum(X_test['Formation_C']==3) 

In [ ]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
n, d, crit = 175, 12, 'gini'
RFC = RandomForestClassifier(n_estimators = n, max_depth = d, criterion = crit, random_state = 0)
RFC.fit(X_train, y_train)

In [ ]:
y_pred = RFC.predict(X_test)
print(classification_report(y_test, y_pred))

In [ ]:
print(confusion_matrix(y_test, y_pred))

## Predict on the remaining wells with Final Drill Date after Dec 2017

In [ ]:
sum(Wells['FinalDrillDate']>pd.to_datetime('2017-12-31'))

In [ ]:
X_FWD = Wells[['KBElevation','TotalDepth','EndDrillDate_InDays', 
        'Profile_C', 'Formation_C', 'Field_C', 'Type_C','MonthsDUC_Status',
           'DUC_CountGroup']][Wells['EndDrillDate_InDays'] > 1460]
X_FWD.shape

In [ ]:
y = X_FWD['MonthsDUC_Status']
yC = X_FWD['DUC_CountGroup']
X_FWD.drop(['MonthsDUC_Status','DUC_CountGroup'], axis =1, inplace = True)
y.shape, yC.shape

In [ ]:
#  Scale X_FWD on fit from X_train
X_FWD = scaler.transform(X_FWD)

In [ ]:
# Predict all forward DUCs & score against actuals
y_FWD = RFC.predict(X_FWD)
print(classification_report(yC, y_FWD))

In [ ]:
print(confusion_matrix(yC, y_FWD))

## Reminder that the confusion matrix is showing the model is not acurately predicting well by well DUC months.  The overall aggregated trend of number of wells with low DUC months and number of wells with high DUC months is better.

## Reset the Early Comp Dates to reverse changes from prior model predictions, then enter the foward predictions.

In [ ]:
 df_R['Early_Comp'] = Wells['Early_Comp']

In [ ]:
# CHANGE RELEVANT EARLY COMPLETION DATES
vals = [0,1,2,3,4,7.45]  #Class 5 has a mean of 7.45 months

for j in range(len(yC)):
    ID = df_R.iloc[yC[j:j+1].index[0]][0]
    
    # CALCULATE THE PREDICTED MONTHS FROM THE CLASSIFICATION 0 TO 5
    val = y_FWD[j]
    pred = vals[val]

    df_R['Early_Comp'][df_R['EPAssetsId']==ID] = df_R['FinalDrillDate'][df_R['EPAssetsId']==ID] + timedelta(days = int(pred*30.45))

In [ ]:
for Month in Months:
    df_R[Month] = 1*( df_R['FinalDrillDate']<=pd.to_datetime(Month) ) - 1*( df_R['Early_Comp']<=pd.to_datetime(Month) )

In [ ]:
DUCs_by_Month_with_y_pred = ['Classification w/o TVD FWD from Dec 31 2017']
for Month in Months:
    DUCs_by_Month_with_y_pred.append(sum(df_R[Month]))

In [ ]:
DUCs_Compare.head(6)

In [ ]:
fig, ax1 = plt.subplots(1, 1, figsize=(15, 8)) 
ax1.plot(DUCs_Compare.iloc[0,1:37], label = DUCs_Compare.iloc[0,0], c = 'm', lw = 1)
ax1.plot(DUCs_Compare.iloc[0,36:], label = DUCs_Compare.iloc[0,0], c = 'm', lw = 3)
#ax1.plot(DUCs_Compare.iloc[1,1:], label = DUCs_Compare.iloc[1,0])
#ax1.plot(DUCs_Compare.iloc[2,1:], label = DUCs_Compare.iloc[2,0])
#ax1.plot(DUCs_Compare.iloc[3,1:], label = DUCs_Compare.iloc[3,0])
#ax1.plot(DUCs_Compare.iloc[4,1:], label = DUCs_Compare.iloc[4,0])
ax1.plot(DUCs_Compare.iloc[5,36:], label = DUCs_Compare.iloc[5,0], ls = '--', lw = 2)

ax1.set_title('Cumulative DUCs by Month: Comparison of Base Case and Forward DUC Prediction Models', fontsize = 16, weight = 'bold')
ax1.xaxis.set_major_locator(ticker.MaxNLocator(11))
ax1.xaxis.set_minor_locator(ticker.AutoMinorLocator())
ax1.grid()
ax1.set_ylim(0, 450)
plt.legend()
plt.show()



## Repeat with Regression

In [ ]:
X = Wells[['KBElevation','TotalDepth','EndDrillDate_InDays', 
        'Profile_C', 'Formation_C', 'Field_C', 'Type_C','MonthsDUC_Status',
           'DUC_CountGroup']][Wells['EndDrillDate_InDays'] <= 1460]
X.shape

In [ ]:
y = X['MonthsDUC_Status']
yC = X['DUC_CountGroup']
X.drop(['MonthsDUC_Status','DUC_CountGroup'], axis =1, inplace = True)
y.shape, yC.shape

In [ ]:
X.head()

In [ ]:
X_train,X_test, y_train, y_test = train_test_split(X, y, test_size=0.10, random_state=0)

In [ ]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
n, d, mf = 175, 26, 'log2'
RFR = RandomForestRegressor(n_estimators = n, max_depth = d, max_features = mf, random_state = 0)
RFR.fit(X_train, y_train)

In [ ]:
y_pred = RFR.predict(X_test)
RFR_R2 = r2_score(y_test, y_pred)
MSE = mean_squared_error(y_test, y_pred)
RMSE = np.sqrt(MSE)
print(n, " ests & " ,d, " max depth,", mf, "max features, R2 for RFR is: ", RFR_R2)
print(n, " ests & ", d, " max depth,", mf, "max features, RMSE RFR is: ", RMSE)

In [ ]:
plt.scatter(y_test, y_pred)
plt.show()

In [ ]:
plt.figure(figsize=(12,5)) 
sns.distplot(y_test, bins = 15, color = 'tab:green', label = 'y_test', kde = False)
sns.distplot(y_pred, bins = 15, color = 'tab:blue', label = 'y-pred', kde = False)


plt.title('Regression Model 2 w/o TVD Histogram y-test & y_pred')
#plt.xlim(0,15)
#plt.yscale('log')
plt.grid()
plt.legend()
plt.show()

## Reset the Early Comp Dates to reverse changes from prior model predictions, then enter the foward predictions.

In [ ]:
X_FWD = Wells[['KBElevation','TotalDepth','EndDrillDate_InDays', 
        'Profile_C', 'Formation_C', 'Field_C', 'Type_C','MonthsDUC_Status',
           'DUC_CountGroup']][Wells['EndDrillDate_InDays'] > 1460]
X_FWD.shape

In [ ]:
y = X_FWD['MonthsDUC_Status']
yC = X_FWD['DUC_CountGroup']
X_FWD.drop(['MonthsDUC_Status','DUC_CountGroup'], axis =1, inplace = True)
y.shape, yC.shape

In [ ]:
#  Scale X_FWD on fit from X_train
X_FWD = scaler.transform(X_FWD)

In [ ]:
y_FWD = RFR.predict(X_FWD)
RFR_R2 = r2_score(y, y_FWD)
MSE = mean_squared_error(y, y_FWD)
RMSE = np.sqrt(MSE)
print(n, " ests & " ,d, " max depth,", mf, "max features, R2 for RFR is: ", RFR_R2)
print(n, " ests & ", d, " max depth,", mf, "max features, RMSE RFR is: ", RMSE)

In [ ]:
 df_R['Early_Comp'] = Wells['Early_Comp']

In [ ]:
# CHANGE RELEVANT EARLY COMPLETION DATES - REGRESSION RESULTS ARE DIRECT MONTHS

for j in range(len(y)):
    ID = df_R.iloc[y[j:j+1].index[0]][0]
    
    # Y_PRED VALUES ARE FORECASTED AS MONTHS NOT CLASSES, SO NO CONVERSION REQUIRED
    pred = y_FWD[j]
    #pred = vals[val]

    df_R['Early_Comp'][df_R['EPAssetsId']==ID] = df_R['FinalDrillDate'][df_R['EPAssetsId']==ID] + timedelta(days = int(pred*30.45))

In [ ]:
for Month in Months:
    df_R[Month] = 1*( df_R['FinalDrillDate']<=pd.to_datetime(Month) ) - 1*( df_R['Early_Comp']<=pd.to_datetime(Month) )

In [ ]:
DUCs_by_Month_with_y_pred = ['Regression w/o TVD FWD from Dec 31 2017']
for Month in Months:
    DUCs_by_Month_with_y_pred.append(sum(df_R[Month]))

In [ ]:
DUCs_Compare.loc[6] = DUCs_by_Month_with_y_pred
DUCs_Compare.head(7)

In [ ]:
fig, ax1 = plt.subplots(1, 1, figsize=(15, 8)) 
ax1.plot(DUCs_Compare.iloc[0,1:37], label = DUCs_Compare.iloc[0,0], c = 'm', lw = 1)
ax1.plot(DUCs_Compare.iloc[0,36:], label = DUCs_Compare.iloc[0,0], c = 'm', lw = 3)
#ax1.plot(DUCs_Compare.iloc[1,1:], label = DUCs_Compare.iloc[1,0])
#ax1.plot(DUCs_Compare.iloc[2,1:], label = DUCs_Compare.iloc[2,0])
#ax1.plot(DUCs_Compare.iloc[3,1:], label = DUCs_Compare.iloc[3,0])
#ax1.plot(DUCs_Compare.iloc[4,1:], label = DUCs_Compare.iloc[4,0])
ax1.plot(DUCs_Compare.iloc[5,36:], label = DUCs_Compare.iloc[5,0], ls = '--', lw = 2)
ax1.plot(DUCs_Compare.iloc[6,36:], label = DUCs_Compare.iloc[5,0], ls = '--', lw = 2)

ax1.set_title('Cumulative DUCs by Month: Comparison of Base Case and Forward DUC Prediction Models', fontsize = 16, weight = 'bold')
ax1.xaxis.set_major_locator(ticker.MaxNLocator(11))
ax1.xaxis.set_minor_locator(ticker.AutoMinorLocator())
ax1.grid()
ax1.set_ylim(0, 550)
plt.legend()
plt.show()



In [ ]:
DUCs_Compare.to_csv('DUCS_Compare.csv')

## Add Operator and Projected Depth to the model

In [ ]:
Raw = pd.read_csv('WellHeader_DatathonRev1.csv')

In [ ]:
Raw[['TotalDepth', 'ProjectedDepth']].info()

In [ ]:
Raw['ProjectedDepth'].fillna(Raw['TotalDepth'], inplace = True)

In [ ]:
Raw[['TotalDepth', 'ProjectedDepth']].info()

In [ ]:
plt.scatter(Raw['TotalDepth'], Raw['ProjectedDepth'])
plt.show

In [ ]:
Raw[['TotalDepth','ProjectedDepth']].describe()

In [ ]:
Wells = pd.merge(Wells, Raw[['EPAssetsId','ProjectedDepth']], left_on = 'EPAssetsId', right_on = 'EPAssetsId', how = 'outer', sort = False)
df_R = pd.merge(df_R, Raw[['EPAssetsId','ProjectedDepth']], left_on = 'EPAssetsId', right_on = 'EPAssetsId', how = 'outer', sort = False)

## New Models, add Current Operator Parent (number, do not need label encoder)

In [ ]:
X = Wells[['CurrentOperatorParent','KBElevation','ProjectedDepth','EndDrillDate_InDays', 
        'Profile_C', 'Formation_C', 'Field_C', 'Type_C','MonthsDUC_Status',
           'DUC_CountGroup']][Wells['EndDrillDate_InDays'] <= 1460]
X.shape

In [ ]:
y = X['MonthsDUC_Status']
yC = X['DUC_CountGroup']
X.drop(['MonthsDUC_Status','DUC_CountGroup'], axis =1, inplace = True)
y.shape, yC.shape

In [ ]:
X.head()

In [ ]:
X_train,X_test, y_train, y_test = train_test_split(X, y, test_size=0.10, random_state=0)

In [ ]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
n, d, mf = 175, 26, 'log2'
RFR_F2 = RandomForestRegressor(n_estimators = n, max_depth = d, max_features = mf, random_state = 0)
RFR_F2.fit(X_train, y_train)

In [ ]:
y_pred = RFR_F2.predict(X_test)
RFR_R2 = r2_score(y_test, y_pred)
MSE = mean_squared_error(y_test, y_pred)
RMSE = np.sqrt(MSE)
print(n, " ests & " ,d, " max depth,", mf, "max features, R2 for RFR is: ", RFR_R2)
print(n, " ests & ", d, " max depth,", mf, "max features, RMSE RFR is: ", RMSE)

In [ ]:
plt.scatter(y_test, y_pred)
plt.show()

In [ ]:
plt.figure(figsize=(12,5)) 
sns.distplot(y_test, bins = 15, color = 'tab:green', label = 'y_test', kde = False)
sns.distplot(y_pred, bins = 15, color = 'tab:blue', label = 'y-pred', kde = False)


plt.title('Regression Model 2 w/o TVD Histogram y-test & y_pred')
#plt.xlim(0,15)
#plt.yscale('log')
plt.grid()
plt.legend()
plt.show()

In [ ]:
 df_R['Early_Comp'] = Wells['Early_Comp']

In [ ]:
X_FWD = Wells[['CurrentOperatorParent','KBElevation','ProjectedDepth','EndDrillDate_InDays', 
        'Profile_C', 'Formation_C', 'Field_C', 'Type_C','MonthsDUC_Status',
           'DUC_CountGroup']][Wells['EndDrillDate_InDays'] > 1460]
X_FWD.shape

In [ ]:
y = X_FWD['MonthsDUC_Status']
yC = X_FWD['DUC_CountGroup']
X_FWD.drop(['MonthsDUC_Status','DUC_CountGroup'], axis =1, inplace = True)
y.shape, yC.shape

In [ ]:
#  Scale X_FWD on fit from X_train
X_FWD = scaler.transform(X_FWD)

In [ ]:
y_FWD = RFR_F2.predict(X_FWD)
RFR_R2 = r2_score(y, y_FWD)
MSE = mean_squared_error(y, y_FWD)
RMSE = np.sqrt(MSE)
print(n, " ests & " ,d, " max depth,", mf, "max features, R2 for RFR is: ", RFR_R2)
print(n, " ests & ", d, " max depth,", mf, "max features, RMSE RFR is: ", RMSE)

In [ ]:
# CHANGE RELEVANT EARLY COMPLETION DATES - REGRESSION RESULTS ARE DIRECT MONTHS
for j in range(len(y)):
    ID = df_R.iloc[y[j:j+1].index[0]][0]
    # Y_PRED VALUES ARE FORECASTED AS MONTHS NOT CLASSES, SO NO CONVERSION REQUIRED
    pred = y_FWD[j]
    #pred = vals[val]
    df_R['Early_Comp'][df_R['EPAssetsId']==ID] = df_R['FinalDrillDate'][df_R['EPAssetsId']==ID] + timedelta(days = int(pred*30.45))

In [ ]:
for Month in Months:
    df_R[Month] = 1*( df_R['FinalDrillDate']<=pd.to_datetime(Month) ) - 1*( df_R['Early_Comp']<=pd.to_datetime(Month) )

In [ ]:
DUCs_by_Month_with_y_pred = ['Regression w Oper & ProjectedDepth FWD from Dec 31 2017']
for Month in Months:
    DUCs_by_Month_with_y_pred.append(sum(df_R[Month]))

In [ ]:
RFR_F2_Results = DUCs_by_Month_with_y_pred


### Classification:  First resplit using yC

In [ ]:
X = Wells[['CurrentOperatorParent','KBElevation','ProjectedDepth','EndDrillDate_InDays', 
        'Profile_C', 'Formation_C', 'Field_C', 'Type_C','MonthsDUC_Status',
           'DUC_CountGroup']][Wells['EndDrillDate_InDays'] <= 1460]
X.shape

In [ ]:
y = X['MonthsDUC_Status']
yC = X['DUC_CountGroup']
X.drop(['MonthsDUC_Status','DUC_CountGroup'], axis =1, inplace = True)
y.shape, yC.shape

In [ ]:
X_train,X_test, y_train, y_test = train_test_split(X,yC, stratify = yC, test_size=0.10, random_state=0)

In [ ]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
n, d, crit = 175, 12, 'gini'
RFC_F2 = RandomForestClassifier(n_estimators = n, max_depth = d, criterion = crit, random_state = 0)
RFC_F2.fit(X_train, y_train)

In [ ]:
y_pred = RFC_F2.predict(X_test)
print(classification_report(y_test, y_pred))

In [ ]:
print(confusion_matrix(y_test, y_pred))

## Predict on the remaining wells with Final Drill Date afte Dec 2017

In [ ]:
sum(Wells['FinalDrillDate']>pd.to_datetime('2017-12-31'))

In [ ]:
X_FWD = Wells[['CurrentOperatorParent','KBElevation','ProjectedDepth','EndDrillDate_InDays', 
        'Profile_C', 'Formation_C', 'Field_C', 'Type_C','MonthsDUC_Status',
           'DUC_CountGroup']][Wells['EndDrillDate_InDays'] > 1460]
X_FWD.shape

In [ ]:
y = X_FWD['MonthsDUC_Status']
yC = X_FWD['DUC_CountGroup']
X_FWD.drop(['MonthsDUC_Status','DUC_CountGroup'], axis =1, inplace = True)
y.shape, yC.shape

In [ ]:
#  Scale X_FWD on fit from X_train
X_FWD = scaler.transform(X_FWD)

In [ ]:
# Predict all forward DUCs & score against actuals
y_FWD = RFC_F2.predict(X_FWD)
print(classification_report(yC, y_FWD))

In [ ]:
print(confusion_matrix(yC, y_FWD))

In [ ]:
df_R['Early_Comp'] = Wells['Early_Comp']

In [ ]:
# CHANGE RELEVANT EARLY COMPLETION DATES - REGRESSION RESULTS ARE DIRECT MONTHS
for j in range(len(y)):
    ID = df_R.iloc[y[j:j+1].index[0]][0]
    # Y_PRED VALUES ARE FORECASTED AS MONTHS NOT CLASSES, SO NO CONVERSION REQUIRED
    pred = y_FWD[j]
    #pred = vals[val]
    df_R['Early_Comp'][df_R['EPAssetsId']==ID] = df_R['FinalDrillDate'][df_R['EPAssetsId']==ID] + timedelta(days = int(pred*30.45))

In [ ]:
for Month in Months:
    df_R[Month] = 1*( df_R['FinalDrillDate']<=pd.to_datetime(Month) ) - 1*( df_R['Early_Comp']<=pd.to_datetime(Month) )

In [ ]:
DUCs_by_Month_with_y_pred = ['Classification w Oper & ProjectedDepth FWD from Dec 31 2017']
for Month in Months:
    DUCs_by_Month_with_y_pred.append(sum(df_R[Month]))

In [ ]:
RFC_F2_Results = DUCs_by_Month_with_y_pred

In [ ]:
DUCs_Compare.loc[7] = RFC_F2_Results
DUCs_Compare.loc[8] = RFR_F2_Results

In [ ]:
DUCs_Compare.head(10)

In [ ]:
fig, ax1 = plt.subplots(1, 1, figsize=(15, 8)) 
ax1.plot(DUCs_Compare.iloc[0,1:37], label = DUCs_Compare.iloc[0,0], c = 'm', lw = 1)
ax1.plot(DUCs_Compare.iloc[0,36:], label = DUCs_Compare.iloc[0,0], c = 'm', lw = 3)
#ax1.plot(DUCs_Compare.iloc[1,1:], label = DUCs_Compare.iloc[1,0])
#ax1.plot(DUCs_Compare.iloc[2,1:], label = DUCs_Compare.iloc[2,0])
#ax1.plot(DUCs_Compare.iloc[3,1:], label = DUCs_Compare.iloc[3,0])
#ax1.plot(DUCs_Compare.iloc[4,1:], label = DUCs_Compare.iloc[4,0])
ax1.plot(DUCs_Compare.iloc[5,36:], label = DUCs_Compare.iloc[5,0], ls = '--', lw = 2)
ax1.plot(DUCs_Compare.iloc[6,36:], label = DUCs_Compare.iloc[6,0], ls = '--', lw = 2)
ax1.plot(DUCs_Compare.iloc[7,36:], label = DUCs_Compare.iloc[7,0], ls = '--', lw = 2)
ax1.plot(DUCs_Compare.iloc[8,36:], label = DUCs_Compare.iloc[8,0], ls = '--', lw = 2)

ax1.set_title('Cumulative DUCs by Month: Comparison of Base Case and DUC Prediction Models', fontsize = 16, weight = 'bold')
ax1.xaxis.set_major_locator(ticker.MaxNLocator(11))
ax1.xaxis.set_minor_locator(ticker.AutoMinorLocator())
ax1.grid()
ax1.set_ylim(0, 550)
plt.legend()
plt.show()


In [ ]:
DUCs_Compare.to_csv('DUCS_Compare.csv')

## Try Modelling on TotalDepth and forecasting on Projected Depth

In [ ]:
X = Wells[['CurrentOperatorParent','KBElevation','TotalDepth','EndDrillDate_InDays', 
        'Profile_C', 'Formation_C', 'Field_C', 'Type_C','MonthsDUC_Status',
           'DUC_CountGroup']][Wells['EndDrillDate_InDays'] <= 1460]
X.shape

In [ ]:
X_FWD = Wells[['CurrentOperatorParent','KBElevation','ProjectedDepth','EndDrillDate_InDays', 
        'Profile_C', 'Formation_C', 'Field_C', 'Type_C','MonthsDUC_Status',
           'DUC_CountGroup']][Wells['EndDrillDate_InDays'] > 1460]
X_FWD.shape

In [ ]:
y = X['MonthsDUC_Status']
yC = X['DUC_CountGroup']
X.drop(['MonthsDUC_Status','DUC_CountGroup'], axis =1, inplace = True)
y.shape, yC.shape

In [ ]:
y_FWD = X_FWD['MonthsDUC_Status']
yC_FWD = X_FWD['DUC_CountGroup']
X_FWD.drop(['MonthsDUC_Status','DUC_CountGroup'], axis =1, inplace = True)
y_FWD.shape, yC_FWD.shape

In [ ]:
X.head(2)

In [ ]:
X_FWD.head(2)

In [ ]:
X_train,X_test, y_train, y_test = train_test_split(X, y, test_size=0.10, random_state=0)

In [ ]:
X_trainC,X_testC, y_trainC, y_testC = train_test_split(X, yC, test_size=0.10, random_state=0)

In [ ]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)
X_FWD = scaler.transform(X_FWD)
X_trainC = scaler.transform(X_trainC)
X_testC = scaler.transform(X_testC)

In [ ]:
n, d, mf = 175, 26, 'log2'
RFR_F3 = RandomForestRegressor(n_estimators = n, max_depth = d, max_features = mf, random_state = 0)
RFR_F3.fit(X_train, y_train)

In [ ]:
y_pred_RFR_F3 = RFR_F3.predict(X_test)
RFR_R2 = r2_score(y_test, y_pred_RFR_F3)
MSE = mean_squared_error(y_test, y_pred_RFR_F3)
RMSE = np.sqrt(MSE)
print(n, " ests & " ,d, " max depth,", mf, "max features, R2 for RFR is: ", RFR_R2)
print(n, " ests & ", d, " max depth,", mf, "max features, RMSE RFR is: ", RMSE)

In [ ]:
plt.scatter(y_test,y_pred_RFR_F3)
plt.show()

In [ ]:
n, d, crit = 175, 12, 'gini'
RFC_F3 = RandomForestClassifier(n_estimators = n, max_depth = d, criterion = crit, random_state = 0)
RFC_F3.fit(X_trainC, y_trainC)

In [ ]:
y_pred_RFC_F3 = RFC_F3.predict(X_testC)
print(classification_report(y_testC, y_pred_RFC_F3))

In [ ]:
print(confusion_matrix(y_testC, y_pred_RFC_F3))

### Predict Forward

In [ ]:
# Predict all forward DUCs & score against actuals
y_FWD_RFC_F3 = RFC_F3.predict(X_FWD)
print(classification_report(yC_FWD, y_FWD_RFC_F3))

In [ ]:
print(confusion_matrix(yC_FWD, y_FWD_RFC_F3))

In [ ]:
y_FWD_RFR_F3 = RFR_F3.predict(X_FWD)
RFR_R2 = r2_score(y_FWD,y_FWD_RFR_F3)
MSE = mean_squared_error(y_FWD,y_FWD_RFR_F3)
RMSE = np.sqrt(MSE)
print(n, " ests & " ,d, " max depth,", mf, "max features, R2 for RFR is: ", RFR_R2)
print(n, " ests & ", d, " max depth,", mf, "max features, RMSE RFR is: ", RMSE)

## Reset Early Comp dates and load results for each model

In [ ]:
df_R['Early_Comp'] = Wells['Early_Comp']

In [ ]:
# CHANGE RELEVANT EARLY COMPLETION DATES - CLASSIFICATION RESULTS ARE CLASSES & HAVE TO BE CONVERTED TO MONTHS
for j in range(len(yC_FWD)):
    ID = df_R.iloc[yC_FWD[j:j+1].index[0]][0]
    # Y_PRED VALUES ARE FORECASTED AS CLASSES, SO CONVERSION REQUIRED
    val = y_FWD_RFC_F3[j]
    pred = vals[val]
    df_R['Early_Comp'][df_R['EPAssetsId']==ID] = df_R['FinalDrillDate'][df_R['EPAssetsId']==ID] + timedelta(days = int(pred*30.45))

In [ ]:
for Month in Months:
    df_R[Month] = 1*( df_R['FinalDrillDate']<=pd.to_datetime(Month) ) - 1*( df_R['Early_Comp']<=pd.to_datetime(Month) )

In [ ]:
DUCs_by_Month_with_y_pred = ['Classification w Oper Trained on TotalDepth, F/C on ProjectedDepth FWD from Dec 31 2017']
for Month in Months:
    DUCs_by_Month_with_y_pred.append(sum(df_R[Month]))

In [ ]:
RFC_F3_Results = DUCs_by_Month_with_y_pred

## Get Results by Formation

In [ ]:
# Load DUCs Compare as a framework for DUC Prediction by Month
DUCs_FWD_by_Formation = pd.read_csv('DUCs_Compare.csv')

In [ ]:
DUCs_FWD_by_Formation.drop('Unnamed: 0', axis = 1, inplace = True)

In [ ]:
DUCs_FWD_by_Formation['Model'] = ['Base - Cardium', 'Base - Duvernay', 'Base - Montney', 'Base - Viking',
                                  'FWD_CLS_F3 - Cardium','FWD_CLS_F3 - Duvernay', 'FWD_CLS_F3 - Montney', 'FWD_CLS_F3 - Viking',
                                 'FWD_REG_F3 - Cardium']#,'FWD_REG_F3 - Duvernay', 'FWD_REG_F3 - Montney', 'FWD_REG_F3 - Viking']

In [ ]:
#Get BASE CASE by Formation
Labels =  ['Base - Cardium', 'Base - Duvernay', 'Base - Montney', 'Base - Viking']
Forms = [0,1,2,3]
for Form in Forms:
    entry = [Labels[Form]]
    for Month in Months:
        entry.append(sum(Wells[Month][Wells['Formation_C'] == Form]))
    DUCs_FWD_by_Formation.loc[Form] = entry
DUCs_FWD_by_Formation.head()

In [ ]:
# ADD CLASSIFICATION BY FORMATION
Labels =  ['FWD_CLS_F3 - Cardium','FWD_CLS_F3 - Duvernay', 'FWD_CLS_F3 - Montney', 'FWD_CLS_F3 - Viking']
Forms = [0,1,2,3]
for Form in Forms:
    entry = [Labels[Form]]
    for Month in Months:
        entry.append(sum(df_R[Month][df_R['Formation_C'] == Form]))
    DUCs_FWD_by_Formation.loc[Form+4] = entry
DUCs_FWD_by_Formation.head(9)

### Save Total DUCs from FWD Classifiction to DUCs_Compare

In [ ]:
DUCs_Compare.loc[9] = RFC_F3_Results

### Get Regression results

In [ ]:
df_R['Early_Comp'] = Wells['Early_Comp']

In [ ]:
# CHANGE RELEVANT EARLY COMPLETION DATES - CLASSIFICATION RESULTS ARE CLASSES & HAVE TO BE CONVERTED TO MONTHS
for j in range(len(y_FWD)):
    ID = df_R.iloc[y_FWD[j:j+1].index[0]][0]
    # Y_PRED VALUES ARE FORECASTED MONTHS SO NO CONVERSION REQUIRED
    pred = y_FWD_RFR_F3[j]
    #pred = vals[val]
    df_R['Early_Comp'][df_R['EPAssetsId']==ID] = df_R['FinalDrillDate'][df_R['EPAssetsId']==ID] + timedelta(days = int(pred*30.45))

In [ ]:
for Month in Months:
    df_R[Month] = 1*( df_R['FinalDrillDate']<=pd.to_datetime(Month) ) - 1*( df_R['Early_Comp']<=pd.to_datetime(Month) )

In [ ]:
DUCs_by_Month_with_y_pred = ['Regression w Oper Trained on TotalDepth, F/C on ProjectedDepth FWD from Dec 31 2017']
for Month in Months:
    DUCs_by_Month_with_y_pred.append(sum(df_R[Month]))

In [ ]:
RFR_F3_Results = DUCs_by_Month_with_y_pred

In [ ]:
DUCs_Compare.loc[10] = RFR_F3_Results

## Get Regression Results by Formation

In [ ]:
# ADD REGRESSION BY FORMATION
Labels =  ['FWD_REG_F3 - Cardium','FWD_REG_F3 - Duvernay', 'FWD_REG_F3 - Montney', 'FWD_REG_F3 - Viking']
Forms = [0,1,2,3]
for Form in Forms:
    entry = [Labels[Form]]
    for Month in Months:
        entry.append(sum(df_R[Month][df_R['Formation_C'] == Form]))
    DUCs_FWD_by_Formation.loc[Form+8] = entry
DUCs_FWD_by_Formation.head(15)

In [ ]:
DUCs_FWD_by_Formation.to_csv('DUCs_FWD_by_Formation.csv')

In [ ]:
DUCs_Compare.to_csv('DUCs_Compare.csv')

### Charts - All Wells & By Formation

In [ ]:
fig, ax1 = plt.subplots(1, 1, figsize=(15, 8)) 
ax1.plot(DUCs_Compare.iloc[0,1:37], label = DUCs_Compare.iloc[0,0], c = 'm', lw = 1)
ax1.plot(DUCs_Compare.iloc[0,36:], label = DUCs_Compare.iloc[0,0], c = 'm', lw = 3)
#ax1.plot(DUCs_Compare.iloc[1,1:], label = DUCs_Compare.iloc[1,0])
#ax1.plot(DUCs_Compare.iloc[2,1:], label = DUCs_Compare.iloc[2,0])
#ax1.plot(DUCs_Compare.iloc[3,1:], label = DUCs_Compare.iloc[3,0])
#ax1.plot(DUCs_Compare.iloc[4,1:], label = DUCs_Compare.iloc[4,0])
ax1.plot(DUCs_Compare.iloc[5,36:], label = DUCs_Compare.iloc[5,0], ls = '--', lw = 2)
ax1.plot(DUCs_Compare.iloc[6,36:], label = DUCs_Compare.iloc[6,0], ls = '--', lw = 2)
ax1.plot(DUCs_Compare.iloc[7,36:], label = DUCs_Compare.iloc[7,0], ls = '--', lw = 2)
ax1.plot(DUCs_Compare.iloc[8,36:], label = DUCs_Compare.iloc[8,0], ls = '--', lw = 2)
ax1.plot(DUCs_Compare.iloc[9,36:], label = DUCs_Compare.iloc[9,0], ls = '--', lw = 2)
ax1.plot(DUCs_Compare.iloc[10,36:], label = DUCs_Compare.iloc[10,0], ls = '--', lw = 2)

ax1.set_title('Cumulative DUCs by Month: Comparison of Base Case and DUC Prediction Models', fontsize = 16, weight = 'bold')
ax1.xaxis.set_major_locator(ticker.MaxNLocator(11))
ax1.xaxis.set_minor_locator(ticker.AutoMinorLocator())
ax1.grid()
#ax1.set_ylim(0, 600)
plt.legend()
plt.show()

## The RF Classification Models under predict, & RF Regression Models over predict.

## Note that the directional trend is followed.  All Wells drilled after Dec 2017 (about 4000 wells) have  predicted number of DUC monts & predicted Completion month.  The DUC information for the prior wells is still in the dataset, although most are not summing to total unless they have never been completed.

In [ ]:
# Function to plot just results by Formation using the DUCs_FWD_by_Formation df results.  Form is 0 to 3

def Form_Plot(df, Form, name, Color):

    fig, ax1 = plt.subplots(1, 1, figsize=(15, 8)) 

    ax1.plot(DUCs_FWD_by_Formation.iloc[Form,1:], label = DUCs_FWD_by_Formation.iloc[Form,0], c = Color, lw = 1)
    ax1.plot(DUCs_FWD_by_Formation.iloc[Form,36:], label = DUCs_FWD_by_Formation.iloc[Form,0], c = Color, lw = 3)

    ax1.plot(DUCs_FWD_by_Formation.iloc[Form + 4,36:], label = DUCs_FWD_by_Formation.iloc[Form+4,0], ls = '--', lw = 2)
    ax1.plot(DUCs_FWD_by_Formation.iloc[Form+8,36:], label = DUCs_FWD_by_Formation.iloc[Form+8,0], ls = '--', lw = 2)

    ax1.set_title(('DUC Inventory for {}: Comparison of Base Case and DUC Prediction Models').format(name), fontsize = 16, weight = 'bold')
    ax1.xaxis.set_major_locator(ticker.MaxNLocator(11))
    ax1.xaxis.set_minor_locator(ticker.AutoMinorLocator())
    ax1.grid()

    plt.legend()
    plt.show()
    return

In [ ]:
Form_Plot(DUCs_FWD_by_Formation, 0, "Cardium", 'b')

In [ ]:
Form_Plot(DUCs_FWD_by_Formation, 1, "Duvernay", 'orange')

In [ ]:
Form_Plot(DUCs_FWD_by_Formation, 2, "Montney", 'g')

In [ ]:
Form_Plot(DUCs_FWD_by_Formation, 3, "Viking", 'r')

## Sum best Models by Formation to get a Total

In [ ]:
Combined_Model = DUCs_FWD_by_Formation.iloc[4,1:] + DUCs_FWD_by_Formation.iloc[9,1:] + DUCs_FWD_by_Formation.iloc[6,1:] + DUCs_FWD_by_Formation.iloc[11,1:] 

In [ ]:
fig, ax1 = plt.subplots(1, 1, figsize=(15, 8)) 
ax1.plot(DUCs_Compare.iloc[0,1:37], label = DUCs_Compare.iloc[0,0], c = 'm', lw = 1)
ax1.plot(DUCs_Compare.iloc[0,36:], label = DUCs_Compare.iloc[0,0], c = 'm', lw = 3)
#ax1.plot(DUCs_Compare.iloc[1,1:], label = DUCs_Compare.iloc[1,0])
#ax1.plot(DUCs_Compare.iloc[2,1:], label = DUCs_Compare.iloc[2,0])
#ax1.plot(DUCs_Compare.iloc[3,1:], label = DUCs_Compare.iloc[3,0])
#ax1.plot(DUCs_Compare.iloc[4,1:], label = DUCs_Compare.iloc[4,0])
ax1.plot(DUCs_Compare.iloc[5,36:], label = DUCs_Compare.iloc[5,0], ls = '--', lw = 2)
ax1.plot(DUCs_Compare.iloc[6,36:], label = DUCs_Compare.iloc[6,0], ls = '--', lw = 2)
ax1.plot(DUCs_Compare.iloc[7,36:], label = DUCs_Compare.iloc[7,0], ls = '--', lw = 2)
ax1.plot(DUCs_Compare.iloc[8,36:], label = DUCs_Compare.iloc[8,0], ls = '--', lw = 2)
ax1.plot(DUCs_Compare.iloc[9,36:], label = DUCs_Compare.iloc[9,0], ls = '--', lw = 2)
ax1.plot(DUCs_Compare.iloc[10,36:], label = DUCs_Compare.iloc[10,0], ls = '--', lw = 2)
ax1.plot(Combined_Model[36:], label = 'Sum of best Models by Formation', ls = '--', lw = 3, c = 'black')

ax1.set_title('Cumulative DUCs by Month: Comparison of Base Case and DUC Prediction Models', fontsize = 16, weight = 'bold')
ax1.xaxis.set_major_locator(ticker.MaxNLocator(11))
ax1.xaxis.set_minor_locator(ticker.AutoMinorLocator())
ax1.grid()
#ax1.set_ylim(0, 600)
plt.legend()
plt.show()

In [ ]:
fig, ax1 = plt.subplots(1, 1, figsize=(15, 8)) 
ax1.plot(DUCs_Compare.iloc[0,1:37], label = DUCs_Compare.iloc[0,0], c = 'm', lw = 1)
ax1.plot(DUCs_Compare.iloc[0,36:], label = DUCs_Compare.iloc[0,0], c = 'm', lw = 3)
#ax1.plot(DUCs_Compare.iloc[1,1:], label = DUCs_Compare.iloc[1,0])
#ax1.plot(DUCs_Compare.iloc[2,1:], label = DUCs_Compare.iloc[2,0])
#ax1.plot(DUCs_Compare.iloc[3,1:], label = DUCs_Compare.iloc[3,0])
#ax1.plot(DUCs_Compare.iloc[4,1:], label = DUCs_Compare.iloc[4,0])
#ax1.plot(DUCs_Compare.iloc[5,36:], label = DUCs_Compare.iloc[5,0], ls = '--', lw = 2)
#ax1.plot(DUCs_Compare.iloc[6,36:], label = DUCs_Compare.iloc[6,0], ls = '--', lw = 2)
#ax1.plot(DUCs_Compare.iloc[7,36:], label = DUCs_Compare.iloc[7,0], ls = '--', lw = 2)
#ax1.plot(DUCs_Compare.iloc[8,36:], label = DUCs_Compare.iloc[8,0], ls = '--', lw = 2)
ax1.plot(DUCs_Compare.iloc[9,36:], label = DUCs_Compare.iloc[9,0], ls = '--', lw = 2)
ax1.plot(DUCs_Compare.iloc[10,36:], label = DUCs_Compare.iloc[10,0], ls = '--', lw = 2)
ax1.plot(Combined_Model[36:], label = 'Sum of best Models by Formation', ls = '--', lw = 3, c = 'black')

ax1.set_title('DUC Inventory by Month: Comparison of Base Case and DUC Prediction Models', fontsize = 18, weight = 'bold')
ax1.xaxis.set_major_locator(ticker.MaxNLocator(11))
ax1.xaxis.set_minor_locator(ticker.AutoMinorLocator())
ax1.grid()
#ax1.set_ylim(0, 600)
plt.legend()
plt.show()

In [ ]:
Combined_Model.to_csv('Combined_Model.csv')

In [ ]:
DUCs_Compare.to_csv('DUCs_Compare.csv')

## The combined model (using the best model by formation of classification or regression) predicts the total DUC inventory reasonably well through 2018 to early 2019.  

## This approach could be taken to predict the gross DUC inventory through a drilling season or a year using planned drilling program information from active operators where that is available from annual reports, announcements, and historic information or drilling licenses.